In [1]:
import os
import json
import pandas as pd
import tensorflow as tf
import numpy as np
import help_functions as hf
from matplotlib import pyplot as plt
from functools import partial
import albumentations as A
AUTOTUNE = tf.data.experimental.AUTOTUNE

hf.setup_gpu(gpu_nr=1)

2 Physical GPUs, 1 Logical GPU


Hyper parameters

In [2]:
with open('training_configurations.json', 'r') as fp:
    config = json.load(fp)[str(11)]
config['nr_classes'] = 5
config['batch_size'] = 512
train, train_df = hf.get_flow(df_file=config['data_folder'] + '/test_df.json.bz2',
                   nr_classes=config['nr_classes'],
                   batch_size=config['batch_size'],
                   image_dimension=config['image_dimension'])
idx_2_label_dict = {v: k for k, v in train.class_indices.items()}

Found 50000 non-validated image filenames belonging to 40 classes.


/home/matvieir/wiki_image_classification/src/classification/help_functions.py:269: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x_labels['labels'] = df['labels'].apply(lambda labels_list: [label for label in labels_list if label in top_classes])


Found 42671 validated image filenames belonging to 5 classes.


Loading datasets and augmentation

In [3]:
train, _ = hf.get_flow(df_file=config['data_folder'] + '/test_df.json.bz2',
                    batch_size=config['batch_size'],
                    nr_classes=config['nr_classes'],
                    image_dimension=config['image_dimension'])

train_ds = tf.data.Dataset.from_generator(
    lambda: train, 
    output_types=(tf.float32, tf.float32), 
    output_shapes=((config['batch_size'], config['image_dimension'], config['image_dimension'], 3), 
                   (config['batch_size'], config['nr_classes'])), 
)
def augment_train_data(ds):

    minority_labels = ['People', 'Culture']

    transforms = A.Compose([
            A.Transpose(always_apply=True),
            A.HorizontalFlip(always_apply=True),
            A.VerticalFlip(always_apply=True),
            A.ShiftScaleRotate(always_apply=True), 
            # A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, always_apply=True), # gives black images
            # A.RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), always_apply=True), # gives black images
            A.CoarseDropout(max_height=4, max_width=4),
            ])
    
    def aug_fn(images, labels):
        # data = {"images": images}
        all_label_names = np.array(list(train.class_indices.keys()))
        aug_imgs = np.zeros((images.shape))
        for img_idx in range(images.shape[0]):
            img = images[img_idx, :, :, :]
            img_label_names = all_label_names[labels[img_idx].astype(bool)]
            if any(label in img_label_names for label in minority_labels):
                img = transforms(image=img)['image']
            aug_imgs[img_idx, :, :, :] = img
            
        # aug_data = transforms(**data)
        # aug_img = aug_data["image"]
        aug_imgs = tf.cast(aug_imgs, tf.float32)
        # aug_img = tf.image.resize(aug_img, size=[config['image_dimension'], config['image_dimension']]) # doesn't do anything. aug_img.shape = (512, 64, 64, 3) before and after
        return aug_imgs

    def process_data(image, label):
        """"
        Here, we we call the augmentation function if the image has any of the minority labels.
        (for the moment I'm just considering [0,0,1,0,0] as a minority label to test)
        """
        print('hola')
        print(f'process_data {label.shape}')
        # if tf.equal(tf.constant(label[2], dtype=tf.float32), tf.constant(1, dtype=tf.float32)):
        #     print('there is people here')
        image = tf.numpy_function(func=aug_fn, inp=[image, label], Tout=tf.float32)
        return image, label
    
    def set_shapes(img, label, img_shape=(config['image_dimension'], config['image_dimension'], 3)):
        print(f'set_shapes {img.shape}') # <unknown>
        img.set_shape(img_shape)
        print(f'set_shapes {img.shape}') # (64,64,3)
        label.set_shape([])
        return img, label
    
    ds_alb = (
            ds
            # .map(partial(process_data))
            .map(partial(process_data), num_parallel_calls=AUTOTUNE)
            .prefetch(AUTOTUNE)
        )
    # ds_alb = (
    #         ds_alb
    #         .map(set_shapes, 
    #         # num_parallel_calls=AUTOTUNE
    #         )
    #     )
    # ds_alb = ds_alb.repeat() # what is the reason of this?
    return ds_alb
train_alb = augment_train_data(train_ds)


Found 50000 non-validated image filenames belonging to 40 classes.


/home/matvieir/wiki_image_classification/src/classification/help_functions.py:269: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x_labels['labels'] = df['labels'].apply(lambda labels_list: [label for label in labels_list if label in top_classes])


Found 42671 validated image filenames belonging to 5 classes.
hola
process_data (512, 5)
